In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Define substep interface
#from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

In [3]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/22 10:26:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Create the streaming_df to read from input directory
streaming_df = spark.read.json("data.json")

# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
#streaming_df.show(truncate=False)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/sinarian/work/ops/test_update/pipeline-sinara_quick_test/data.json.

In [ ]:
from pyspark.sql.functions import arrays_zip, explode

flatten = streaming_df.withColumn("ex", explode(arrays_zip("Confidence", "Sequence", "TimeStamp", "Value"))) \
    .select("Status", "TagName",  "TagNumber", "Units", "ex.Confidence", "ex.Sequence", "ex.TimeStamp", "ex.Value")

flatten.show()

In [ ]:
flatten.printSchema()
#flattened_df.show(2, truncate= False)
flatten.write.json("./data_lines.json", mode= "overwrite")

In [ ]:
import glob
data_lines_json = glob.glob('./data_lines.json/*.json')[0]
with open(data_lines_json, "r") as f:
    contents = f.readlines()

    
index = 1000
contents.insert(index, '{"GUID":"321"},')

with open("./data_augmented.json", "w") as f:
    contents = "".join(contents)
    f.write(contents)

In [ ]:
augmented = spark.read.json("./data_augmented.json")


spark.sql("set spark.sql.streaming.schemaInference=true")
augmented_stream = spark.readStream.json("./data_augmented.json")

#augmented_stream = spark.readStream.json("./data_augmented.json", augmented.schema)

augmented.printSchema()

augmented_stream.printSchema()

In [ ]:
#8 stop spark
SinaraSpark.stop_session()